In [ ]:
!pip -q install TTS soundfile

from huggingface_hub import snapshot_download
!git clone https://huggingface.co/coqui/XTTS-v2

from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import soundfile as sf
import os

config = XttsConfig()
config.load_json("/content/XTTS-v2/config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/content/XTTS-v2/", eval=True)
model.cuda()

def process_tts(input_text, speaker_audio, lang, output_file):
    outputs = model.synthesize(
      input_text,
      config,
      speaker_wav=speaker_audio,
      gpt_cond_len=3,
      language=lang,
    )
    audio_data = outputs["wav"]
    sample_rate = 24000
    sf.write(output_file, audio_data, sample_rate)
    print("Audio saved successfully!")

In [ ]:
text_en = """The quick brown fox jumps over the lazy dog near the idyllic riverbank, causing ripples in the water's mirrored surface.
In the silence of the ancient library, amidst the scent of old books, a whisper echoed: "Preserve the knowledge, for it holds the key to the future."""

reference_audio = "/content/XTTS-v2/samples/female_en.wav"
process_tts(text_en, reference_audio, "en", "audio_output.wav")